In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

product_url = "https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html?piid=1702143331&auctionId=9bdbad2e-226f-4574-890f-49c7ace6183d&adTypeId=1"
payload = {
    "source": "universal",
    "url": product_url,
    "user_agent_type": "desktop_safari",
    "geo_location": "United States",
    "render": "html",
    "browser_instructions": [
        {
            "type": "wait_for_element",
            "selector": {
                "type": "css",
                "value": "div.SFPrice span.oakhm64z_6112"
            },
            "timeout_s": 10
        }
    ]
}

username = "joe528491_EoLxB"
password = "Adej24972497_02"

response = requests.post(
    "https://realtime.oxylabs.io/v1/queries",
    auth=(username, password),
    json=payload,
    timeout=180
)
print(response.status_code)

content = response.json()["results"][0]["content"]
soup = BeautifulSoup(content, "html.parser")

title = soup.find("h1", {"data-hb-id": "Heading"}).text
price = soup.find("div", {"class": "SFPrice"}).find("span", {"class": "oakhm64z_6112"}).text
rating = soup.find("span", {"class": "ProductRatingNumberWithCount-rating"}).text

data = [{
   "Product Title": title,
   "Price": price,
   "Rating": rating,
   "Link": product_url,
}]
df = pd.DataFrame(data)
# df.to_csv("product_data.csv", index=False)
# df.to_json("product_data.json", orient="records")

401


KeyError: 'results'

In [15]:
import requests
from bs4 import BeautifulSoup

# Your Oxylabs API credentials
USERNAME = "joe528491_EoLxB"
PASSWORD = "Adej24972497_01"

# Set up payload
payload = {
    "source": "universal_ecommerce",  # 或是 "universal_ecommerce"，看你要不要加 render
    "url": "https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html",
    "user_agent_type": "desktop_safari",
    "geo_location": "United States",
    "render": "html"  # html代表要完整渲染
}

# Send POST request
response = requests.post(
    "https://realtime.oxylabs.io/v1/queries",
    auth=(USERNAME, PASSWORD),
    json=payload,
    timeout=180
)

# Check response
if response.status_code == 200:
    result = response.json()
    html_content = result["results"][0]["content"]

    # Optional: parse with BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Example: find price
    price_span = soup.find("span", {"data-test-id": "PriceDisplay"})
    if price_span:
        price = price_span.get_text(strip=True)
        print(f"💲 Product Price: {price}")
    else:
        print("⚠️ Price not found.")

else:
    print(f"❌ Request failed with status code {response.status_code}")
    print(response.text)


💲 Product Price: $539.99


In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time

# Your Oxylabs API credentials
USERNAME = "joe528491_EoLxB"
PASSWORD = "Adej24972497_01"

# Load the URLs from CSV
df = pd.read_csv(r"..\dat\wayfair_bs4_products.csv")
urls = df['url'].dropna().head(10).tolist()

# Create a list to store the results
results = []

# Loop over each URL
for idx, url in enumerate(urls, 1):
    print(f"[INFO] Scraping ({idx}/{len(urls)}): {url}")
    
    payload = {
        "source": "universal_ecommerce",
        "url": url,
        "user_agent_type": "desktop_safari",
        "geo_location": "United States",
        "render": "html"
    }

    try:
        response = requests.post(
            "https://realtime.oxylabs.io/v1/queries",
            auth=(USERNAME, PASSWORD),
            json=payload,
            timeout=180
        )

        if response.status_code == 200:
            result_json = response.json()
            html_content = result_json["results"][0]["content"]

            soup = BeautifulSoup(html_content, "html.parser")
            price_span = soup.find("span", {"data-test-id": "PriceDisplay"})

            if price_span:
                price = price_span.get_text(strip=True)
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                print(f"✅ Found price: {price}")
                results.append({
                    "timestamp": timestamp,
                    "url": url,
                    "price": price
                })
            else:
                print("⚠️ Price not found.")
                results.append({
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "url": url,
                    "price": None
                })
        else:
            print(f"❌ Request failed with status code {response.status_code}")
            results.append({
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "url": url,
                "price": None
            })

    except Exception as e:
        print(f"⚠️ Error: {e}")
        results.append({
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "url": url,
            "price": None
        })

    time.sleep(2)  # Optional: small delay between requests to be gentle

# Convert results into a DataFrame
final_df = pd.DataFrame(results)

# Preview
print(final_df)

# Save results (optional, remove comments if needed)
# final_df.to_csv(r"..\dat\wayfair_price_tracking.csv", index=False)


[INFO] Scraping (1/10): https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html?piid=1702143331&auctionId=9bdbad2e-226f-4574-890f-49c7ace6183d&adTypeId=1
✅ Found price: $539.99
[INFO] Scraping (2/10): https://www.wayfair.com/furniture/pdp/august-grove-dediu-farmhouse-computer-desk-with-charging-station-wood-corner-desk-with-monitor-stand-file-drawer-w112473542.html?piid=67687832&auctionId=9bdbad2e-226f-4574-890f-49c7ace6183d&adTypeId=1
✅ Found price: $259.99
[INFO] Scraping (3/10): https://www.wayfair.com/furniture/pdp/inbox-zero-koree-standing-height-adjustable-desks-w111003401.html?piid=1458745315%2C1458745313
⚠️ Price not found.
[INFO] Scraping (4/10): https://www.wayfair.com/furniture/pdp/george-oliver-caidence-solid-wood-desk-w005949248.html?piid=553739021%2C553739020
⚠️ Price not found.
[INFO] Scraping (5/10): https://www.wayfair.com/furniture/pdp/wade-logan-channay-desk-w005849507.html?piid=564543439&auctio

In [17]:
final_df

,timestamp,url,price
0,2025-04-27 21:40:51,https://www.wayfair.com/furniture/pdp/hokku-de...,$539.99
1,2025-04-27 21:41:19,https://www.wayfair.com/furniture/pdp/august-g...,$259.99
2,2025-04-27 21:41:57,https://www.wayfair.com/furniture/pdp/inbox-ze...,None
3,2025-04-27 21:42:34,https://www.wayfair.com/furniture/pdp/george-o...,None
4,2025-04-27 21:43:59,https://www.wayfair.com/furniture/pdp/wade-log...,None
5,2025-04-27 21:44:29,https://www.wayfair.com/furniture/pdp/ebern-de...,$203.99
6,2025-04-27 21:45:09,https://www.wayfair.com/furniture/pdp/hokku-de...,$539.99
7,2025-04-27 21:45:40,https://www.wayfair.com/furniture/pdp/latitude...,$193.99
8,2025-04-27 21:46:12,https://www.wayfair.com/furniture/pdp/ebern-de...,$299.99
9,2025-04-27 21:46:58,https://www.wayfair.com/furniture/pdp/inbox-ze...,$79.99


In [21]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time

# Your Oxylabs API credentials
USERNAME = "joe528491_EoLxB"
PASSWORD = "Adej24972497_01"

# Load the URLs from CSV
df = pd.read_csv(r"..\dat\wayfair_bs4_products.csv")
urls = df['url'].dropna().head(10).tolist()

# Create a list to store the results
results = []

def get_price(url):
    """Send API request and extract price from the page."""
    payload = {
        "source": "universal_ecommerce",
        "url": url,
        "user_agent_type": "desktop_safari",
        "geo_location": "United States",
        "render": "html"
    }

    try:
        response = requests.post(
            "https://realtime.oxylabs.io/v1/queries",
            auth=(USERNAME, PASSWORD),
            json=payload,
            timeout=180
        )

        if response.status_code == 200:
            result_json = response.json()
            html_content = result_json["results"][0]["content"]
            soup = BeautifulSoup(html_content, "html.parser")
            price_span = soup.find("span", {"data-test-id": "PriceDisplay"})

            if price_span:
                return price_span.get_text(strip=True)
            else:
                return None
        else:
            print(f"❌ Request failed with status code {response.status_code}")
            return None

    except Exception as e:
        print(f"⚠️ Error during request: {e}")
        return None

# Initial scrape
for idx, url in enumerate(urls, 1):
    print(f"[INFO] Scraping ({idx}/{len(urls)}): {url}")
    price = get_price(url)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results.append({"timestamp": timestamp, "url": url, "price": price})
    time.sleep(2)

# Retry logic
max_retries = 3
for attempt in range(1, max_retries + 1):
    not_found = [r for r in results if r["price"] is None]

    if not not_found:
        print(f"[SUCCESS] All prices found after {attempt - 1} retries.")
        break

    print(f"[RETRY] Attempt {attempt}: {len(not_found)} products missing prices, retrying...")

    for item in not_found:
        price = get_price(item["url"])
        if price:
            item["price"] = price
            item["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        time.sleep(2)

# Convert final results into DataFrame
final_df = pd.DataFrame(results)

# Preview
print(final_df)

# Save results (optional)
final_df.to_csv(r"..\dat\wayfair_price_tracking.csv", index=False)


[INFO] Scraping (1/10): https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html?piid=1702143331&auctionId=9bdbad2e-226f-4574-890f-49c7ace6183d&adTypeId=1
[INFO] Scraping (2/10): https://www.wayfair.com/furniture/pdp/august-grove-dediu-farmhouse-computer-desk-with-charging-station-wood-corner-desk-with-monitor-stand-file-drawer-w112473542.html?piid=67687832&auctionId=9bdbad2e-226f-4574-890f-49c7ace6183d&adTypeId=1


KeyboardInterrupt: 

In [22]:
final_df.to_csv(r"..\dat\wayfair_price_tracking.csv", index=False)

In [20]:
final_df

,timestamp,url,price
0,2025-04-27 21:54:42,https://www.wayfair.com/furniture/pdp/hokku-de...,$539.99
1,2025-04-27 21:55:19,https://www.wayfair.com/furniture/pdp/august-g...,$259.99
2,2025-04-27 21:56:50,https://www.wayfair.com/furniture/pdp/inbox-ze...,None
3,2025-04-27 21:58:54,https://www.wayfair.com/furniture/pdp/george-o...,None
4,2025-04-27 22:00:55,https://www.wayfair.com/furniture/pdp/wade-log...,None
5,2025-04-27 22:01:48,https://www.wayfair.com/furniture/pdp/ebern-de...,$203.99
6,2025-04-27 22:02:27,https://www.wayfair.com/furniture/pdp/hokku-de...,$539.99
7,2025-04-27 22:02:57,https://www.wayfair.com/furniture/pdp/latitude...,$193.99
8,2025-04-27 22:03:24,https://www.wayfair.com/furniture/pdp/ebern-de...,$299.99
9,2025-04-27 22:04:08,https://www.wayfair.com/furniture/pdp/inbox-ze...,$77.99


In [13]:
print(result)

{'results': [{'content': '', 'created_at': '2025-04-28 00:58:15', 'updated_at': '2025-04-28 00:59:08', 'page': 1, 'url': 'https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html', 'job_id': '7322423874488919042', 'is_render_forced': False, 'status_code': 613}], 'job': {'callback_url': None, 'client_id': 109838, 'context': [{'key': 'force_headers', 'value': False}, {'key': 'force_cookies', 'value': False}, {'key': 'hc_policy', 'value': True}, {'key': 'successful_status_codes', 'value': []}, {'key': 'follow_redirects', 'value': None}, {'key': 'cookies', 'value': []}, {'key': 'headers', 'value': []}, {'key': 'session_id', 'value': None}, {'key': 'http_method', 'value': 'get'}, {'key': 'content', 'value': None}, {'key': 'store_id', 'value': None}, {'key': 'proxy_location', 'value': None}, {'key': 'delivery_location', 'value': None}, {'key': 'fulfillment_type', 'value': None}], 'created_at': '2025-04-28 00:58:15', 'dom

In [14]:
print(response.json())


{'results': [{'content': '', 'created_at': '2025-04-28 00:58:15', 'updated_at': '2025-04-28 00:59:08', 'page': 1, 'url': 'https://www.wayfair.com/furniture/pdp/hokku-designs-63-l-shaped-office-executive-standing-desk-with-file-cabinet-w112715089.html', 'job_id': '7322423874488919042', 'is_render_forced': False, 'status_code': 613}], 'job': {'callback_url': None, 'client_id': 109838, 'context': [{'key': 'force_headers', 'value': False}, {'key': 'force_cookies', 'value': False}, {'key': 'hc_policy', 'value': True}, {'key': 'successful_status_codes', 'value': []}, {'key': 'follow_redirects', 'value': None}, {'key': 'cookies', 'value': []}, {'key': 'headers', 'value': []}, {'key': 'session_id', 'value': None}, {'key': 'http_method', 'value': 'get'}, {'key': 'content', 'value': None}, {'key': 'store_id', 'value': None}, {'key': 'proxy_location', 'value': None}, {'key': 'delivery_location', 'value': None}, {'key': 'fulfillment_type', 'value': None}], 'created_at': '2025-04-28 00:58:15', 'dom